In [ ]:
import asyncio
import websockets
import json

# Получение цены тикера

In [3]:
import requests

def get_binance_price(symbol="BTCUSDT"):
    url = f"https://api.binance.com/api/v3/ticker/price?symbol={symbol}"
    response = requests.get(url)
    data = response.json()
    return data

print(get_binance_price("BTCUSDT"))
print(get_binance_price("ETHUSDT"))
print(get_binance_price("SOLUSDT"))

{'symbol': 'BTCUSDT', 'price': '104176.17000000'}
{'symbol': 'ETHUSDT', 'price': '2500.50000000'}
{'symbol': 'SOLUSDT', 'price': '173.89000000'}


# Получение цен всех тикеров

In [8]:
def get_all_prices():
    url = "https://api.binance.com/api/v3/ticker/price"
    response = requests.get(url)
    return response.json()

prices = get_all_prices()
print(prices)

[{'symbol': 'ETHBTC', 'price': '0.02406000'}, {'symbol': 'LTCBTC', 'price': '0.00097100'}, {'symbol': 'BNBBTC', 'price': '0.00630700'}, {'symbol': 'NEOBTC', 'price': '0.00006690'}, {'symbol': 'QTUMETH', 'price': '0.00101100'}, {'symbol': 'EOSETH', 'price': '0.00036350'}, {'symbol': 'SNTETH', 'price': '0.00001700'}, {'symbol': 'BNTETH', 'price': '0.00020260'}, {'symbol': 'BCCBTC', 'price': '0.00000000'}, {'symbol': 'GASBTC', 'price': '0.00003350'}, {'symbol': 'BNBETH', 'price': '0.26210000'}, {'symbol': 'BTCUSDT', 'price': '104187.08000000'}, {'symbol': 'ETHUSDT', 'price': '2507.93000000'}, {'symbol': 'HSRBTC', 'price': '0.00000000'}, {'symbol': 'OAXETH', 'price': '0.00000000'}, {'symbol': 'DNTETH', 'price': '0.00000000'}, {'symbol': 'MCOETH', 'price': '0.00000000'}, {'symbol': 'ICNETH', 'price': '0.00000000'}, {'symbol': 'MCOBTC', 'price': '0.00000000'}, {'symbol': 'WTCBTC', 'price': '0.00000024'}, {'symbol': 'WTCETH', 'price': '0.00000000'}, {'symbol': 'LRCBTC', 'price': '0.00000115'}

# Стрим тикеров через websockets
https://developers.binance.com/docs/binance-spot-api-docs/web-socket-streams

In [ ]:
symbols = ["btcusdt", "ethusdt", "solusdt"]
streams = "/".join([f"{s}@ticker" for s in symbols])
url = f"wss://stream.binance.com:9443/stream?streams={streams}"

async def listen():
    async with websockets.connect(url) as websocket:
        while True:
            data = await websocket.recv()
            parsed = json.loads(data)
            symbol = parsed["data"]["s"]
            price = parsed["data"]["c"]
            print(f"{symbol}: {price}")

asyncio.run(listen())

#### Формат ответа

```json
{
    'stream': 'solusdt@ticker',
    'data': {
        "e": "24hrTicker",  // Event type
        "E": 1672515782136, // Event time
        "s": "BNBBTC",      // Symbol
        "p": "0.0015",      // Price change
        "P": "250.00",      // Price change percent
        "w": "0.0018",      // Weighted average price
        "x": "0.0009",      // First trade(F)-1 price (first trade before the 24hr rolling window)
        "c": "0.0025",      // Last price
        "Q": "10",          // Last quantity
        "b": "0.0024",      // Best bid price
        "B": "10",          // Best bid quantity
        "a": "0.0026",      // Best ask price
        "A": "100",         // Best ask quantity
        "o": "0.0010",      // Open price
        "h": "0.0025",      // High price
        "l": "0.0010",      // Low price
        "v": "10000",       // Total traded base asset volume
        "q": "18",          // Total traded quote asset volume
        "O": 0,             // Statistics open time
        "C": 86400000,      // Statistics close time
        "F": 0,             // First trade ID
        "L": 18150,         // Last trade Id
        "n": 18151          // Total number of trades
    }
}
```

# Получаем минутные свечи 

In [ ]:
symbols = ["btcusdt", "ethusdt", "solusdt"]
streams = "/".join([f"{s}@kline_1m" for s in symbols])
url = f"wss://stream.binance.com:9443/stream?streams={streams}"

async def listen():
    async with websockets.connect(url) as ws:
        while True:
            msg = await ws.recv()
            parsed = json.loads(msg)
            data = parsed["data"]["k"] # kline data
            if data["x"]:  # x=True означает, что свеча закрыта (complete)
                symbol = data["s"]
                open_price = float(data["o"])
                close_price = float(data["c"])
                print(symbol, open_price, close_price)

asyncio.run(listen())

#### Формат ответа
```json
{
    'stream': 'solusdt@kline_1m',
    'data': {
        "e": "kline",         // Event type
        "E": 1672515782136,   // Event time
        "s": "BNBBTC",        // Symbol
        "k": {
            "t": 1672515780000, // Kline start time
            "T": 1672515839999, // Kline close time
            "s": "BNBBTC",      // Symbol
            "i": "1m",          // Interval
            "f": 100,           // First trade ID
            "L": 200,           // Last trade ID
            "o": "0.0010",      // Open price
            "c": "0.0020",      // Close price
            "h": "0.0025",      // High price
            "l": "0.0015",      // Low price
            "v": "1000",        // Base asset volume
            "n": 100,           // Number of trades
            "x": false,         // Is this kline closed?
            "q": "1.0000",      // Quote asset volume
            "V": "500",         // Taker buy base asset volume
            "Q": "0.500",       // Taker buy quote asset volume
            "B": "123456"       // Ignore
        }
    }
}
```

In [1]:
import pandas as pd

print(pd.read_parquet('/home/koshkidadanet/My Files/FireflyX/03_05_1m.parquet').head())

df = pd.read_parquet('/home/koshkidadanet/My Files/FireflyX/03_05_1m.parquet')

     ticker            datetime    open    high     low   close    volume
0  SOL/USDT 2025-03-01 00:00:00  148.18  148.23  147.85  147.85  5974.582
1  SOL/USDT 2025-03-01 00:01:00  147.86  147.98  147.68  147.77  5389.290
2  SOL/USDT 2025-03-01 00:02:00  147.77  147.77  147.42  147.60  6629.311
3  SOL/USDT 2025-03-01 00:03:00  147.59  147.60  147.41  147.45  1836.477
4  SOL/USDT 2025-03-01 00:04:00  147.45  147.59  147.24  147.26  3683.697


In [2]:
df['date'] = df['datetime'].dt.date

In [5]:
plot_df = df.loc[
    # (df.ticker == 'XRP/USDT') & (df.datetime.between('2025-03-02 14:00:00', '2025-03-02 18:00:00'))
    # |(df.ticker == 'SOL/USDT') & (df.datetime.between('2025-03-02 14:30:00', '2025-03-02 18:30:00'))
    (df.ticker == 'ADA/USDT') & (df.datetime.between('2025-03-02 15:00:00', '2025-03-02 19:00:00'))
]

# 2025-03-02

In [67]:
import json

tickers = plot_df['ticker'].unique()

# Разбиваем по тикерам
ticker_dfs = {
    ticker: plot_df[plot_df['ticker'] == ticker].sort_values('datetime').reset_index(drop=True)
    for ticker in tickers
}

lengths = [len(df_) for df_ in ticker_dfs.values()]
if len(set(lengths)) != 1:
    raise ValueError("Количество свечей у разных тикеров не совпадает")

n_rows = lengths[0]
result = []

for i in range(n_rows):
    for ticker in tickers:
        row = ticker_dfs[ticker].iloc[i]
        result.append({
            's': ticker.replace('/', '').lower(),
            'o': float(row['open']),
            'c': float(row['close']),
            'datetime': row['datetime'].isoformat()
        })

# Сохраняем в JSON
with open('candles.json', 'w') as f:
    json.dump(result, f, indent=2)

# Загрузка при необходимости
with open('candles.json', 'r') as f:
    loaded_result = json.load(f)

In [8]:
import plotly.graph_objs as go

fig = go.Figure(
    data=[go.Candlestick(
        x=plot_df['datetime'],
        open=plot_df['open'],
        high=plot_df['high'],
        low=plot_df['low'],
        close=plot_df['close'],
        name='OHLC'
    )]
)

fig.update_layout(
    title='BTC/USDT Свечной график',
    xaxis_title='Время',
    yaxis_title='Цена (USDT)'
)

fig.show()